In [ ]:
import nltk
import pandas as pd
import string
from nltk.probability import FreqDist
import numpy as np
import matplotlib.pyplot as plt
import scipy

In [ ]:
# import the Fake.csv and True.csv files
fake = pd.read_csv('Fake.csv')
true = pd.read_csv('True.csv')

In [ ]:
nltk.download('punkt')
true_string = ''.join(true["text"])
true_tokenized = nltk.word_tokenize(true_string)

In [ ]:
fake_string = ''.join(fake["text"])
fake_tokenized = nltk.word_tokenize(fake_string)

In [ ]:
true_tokenized = [word.lower() for word in true_tokenized if word.isalnum() or (not "." and not "," and not "?" and not "!")]

In [ ]:
fake_tokenized = [word.lower() for word in fake_tokenized if word.isalnum() or (not "." and not "," and not "?" and not "!")]

In [ ]:
fdist_true = FreqDist(true_tokenized)
print("True: " + str(fdist_true.most_common(100)))

In [ ]:
fdist_fake = FreqDist(fake_tokenized)
print("Fake: " + str(fdist_fake.most_common(100)))

In [ ]:
fig = plt.figure(figsize = (100,20))
fdist_true.plot(100,cumulative=False)
plt.show()
fig.savefig('true_freq.png', bbox_inches = "tight")

In [ ]:
fig = plt.figure(figsize = (100,20))
fdist_fake.plot(100,cumulative=False)
plt.show()
fig.savefig('fake_freq.png', bbox_inches = "tight")

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
true_tokenized = [word for word in true_tokenized if word not in stop_words]
fake_tokenized = [word for word in fake_tokenized if word not in stop_words]

In [ ]:
fdist_true = FreqDist(true_tokenized)
print("True: " + str(fdist_true.most_common(100)))

In [ ]:
fdist_fake = FreqDist(fake_tokenized)
print("Fake: " + str(fdist_fake.most_common(100)))

In [ ]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
true_tokenized = [lemmatizer.lemmatize(word) for word in true_tokenized]
fake_tokenized = [lemmatizer.lemmatize(word) for word in fake_tokenized]

In [ ]:
fdist_true = FreqDist(true_tokenized)
print("True: " + str(fdist_true.most_common(100)))

In [ ]:
fdist_fake = FreqDist(fake_tokenized)
print("Fake: " + str(fdist_fake.most_common(100)))

In [ ]:
print("type: " + str(type(true_counts)) + " size: " + str(true_counts.shape))

In [ ]:
size_of_true = true_counts.shape[0]
size_of_fake = fake_counts.shape[0]
sentiment = [0 for i in range(size_of_true)]
sentiment.extend(1 for i in range(size_of_fake))
print("Size of True: " + str(size_of_true) + "\nSize of Fake: " +
      str(size_of_fake) + "\nSize of Sentiment: " + str(len(sentiment)))


In [ ]:
true['label']=0
fake['label']=1
true_pandas = pd.DataFrame(true.to_numpy())
fake_pandas = pd.DataFrame(fake.to_numpy())
data = pd.concat([true_pandas,fake_pandas])
print(data[4])


In [55]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
#tokenizer to remove unwanted elements from out data like symbols and numbers
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(lowercase=True,stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
data_clean = cv.fit_transform(data[1])

In [61]:
# Train model
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_clean, data[4].astype("int"), test_size=0.2, random_state=0)

In [62]:
from sklearn.naive_bayes import MultinomialNB
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Generation Using Multinomial Naive Bayes
clf = MultinomialNB().fit(X_train, y_train)
predicted= clf.predict(X_test)
print("MultinomialNB Accuracy:",metrics.accuracy_score(y_test, predicted))

MultinomialNB Accuracy: 0.9552338530066815


In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer()
text_tf= tf.fit_transform(data[1])

In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    text_tf, data[4].astype("int"), test_size=0.3, random_state=123)

In [66]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
# Model Generation Using Multinomial Naive Bayes
clf = MultinomialNB().fit(X_train, y_train)
predicted= clf.predict(X_test)
print("MultinomialNB Accuracy:",metrics.accuracy_score(y_test, predicted))

MultinomialNB Accuracy: 0.9398663697104677


In [67]:
# Classification using Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    text_tf, data[4].astype("int"), test_size=0.3, random_state=123)

clf = LogisticRegression().fit(X_train, y_train)
predicted= clf.predict(X_test)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))


Logistic Regression Accuracy: 0.985894580549369


In [68]:
# Classification using SVM
from sklearn.svm import SVC
X_train, X_test, y_train, y_test = train_test_split(
    text_tf, data[4].astype("int"), test_size=0.3, random_state=123)

clf = SVC().fit(X_train, y_train)
predicted= clf.predict(X_test)
print("SVM Accuracy:",metrics.accuracy_score(y_test, predicted))